# TP Spark - Utilisation de Spark Streaming

## ETAPE 1 : Cloner ce notebook
* Cliquer sur le bouton "clone this note", ci-dessus (5ème bouton à côté de "TP-Spark-Streaming")

Dans ce TP nous allons utiliser des datasets issus du contrôle aérien des États Unis. Dans ce cas, nous avons des données concernant différents vols au sein des États Unis en 2014. Ceux-ci listent les différents vols, leur lieux de départ et de destination, la distance parcourue, la date et aussi les retards. 

Un fichier en format json a déjà été enregistré sur HDFS, nous allons définir le schéma et charger ce fichier en mémoire.

In [2]:
from pyspark.sql.types import *
departureDelays_geo_schema = StructType([StructField("tripid",IntegerType(),True),
                    StructField("localdate",TimestampType(),True),
                    StructField("delay",IntegerType(),True),
                    StructField("distance",IntegerType(),True),
                    StructField("src",StringType(),True),
                    StructField("dst",StringType(),True),
                    StructField("city_src",StringType(),True),
                    StructField("city_dst",StringType(),True),
                    StructField("state_src",StringType(),True),
                    StructField("state_dst",StringType(),True)])
    
    

departureDelays_static = (spark.read.schema(departureDelays_geo_schema).json("/user/zeppelin/data-source/departureDelays_all.json"))


La variable `departureDelays_static`, comme son nom l'indique, est un DataFrame statique (type *batch*). 

In [4]:
z.show(departureDelays_static)

Maintenant nous voulons obtenir le nombre de vols par état (`departureDelays_static.state_src`), regroupés par des "fenêtres" de temps de 5h.
Pour cela, utilisez la fonction `window()` (un exemple est disponible à https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=groupby#pyspark.sql.functions.window ).



In [6]:
from pyspark.sql.functions import *
# votre code ici

In [7]:
z.show(staticCountsDF)

In [8]:
# Maintenant on va garder ce DataFrame en mémoire cache, et créer une table qu'on peut requêter avec du SQL
staticCountsDF.cache()


staticCountsDF.createOrReplaceTempView("static_counts")

la prochaine ligne fait directement du SQL. Ici, on récupère les données précédentes, en faisant la transformation de la date (on extrait la date et l'heure finale de chaque période).

In [10]:
%%sql
select state_src, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, state_src


L'example précédent a utilise des DataFrames statiques. Dans la suite, nous allons faire l'usage du Structured Streaming de Spark.

Pour simuler un stream, nous allons prendre un répertoire avec plusieurs fichiers. Ces fichiers sont en fait le même fichier source précédent, découpé en 200 fichiers différents de manière plus ou moins aléatoire.


Afin de lire ce stream, nous allons "connecter" le stream à une source fichier. Nous allons "tricher" un peu, en obligeant la lecture d'un fichier à la fois... Cela nous permettra (on espère) de voir l'évolution des tables.

In [14]:
from pyspark.sql.functions import *

inputPath = "/user/zeppelin/data-source/departureDelays_json"

# On utilise la même définition est schéma de staticInputDF, seulement on fera l'usage de `readStream` au lieu de `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(departureDelays_geo_schema) # Le schéma des fichiers JSON
    .option("maxFilesPerTrigger", 1)  # Traite les fichiers comme un stream, en lisant un à la fois
    .json(inputPath)
)



In [15]:
# Est-ce vraiment un stream ? La sortie de `isStreaming` indique quel est le status de ce DataFrame
streamingInputDF.isStreaming

In [16]:
# On prépare la même requête que précédemment. Attention, le stream n'a pas encore démarré, on a juste mis en place la source et une opération
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.state_src, 
      window(streamingInputDF.localdate, "5 hours"))    
    .count()
)


### Le moment est arrivé

Et voilà, c'est le moment de démarrer le traitement du stream. Dans le paragraphe suivant vous allez déclancher un query (un tableau dynamique), qui sera alimentée par le stream (les fichiers).
Pour observer cela, on a fait plusiers requêtes SQL intercalées par des sleep. Les résultats affichés dans les paragraphes suivants devraient être modifiées au fil du temps, indiquant l'annexation de nouvelles informations selon l'arrivage des données du stream.  

 *Aucune garantie sur le résultat :) Ça a marché pour moi mais tout dépend de la vitesse de traitement des fichiers (si tout est fait rapidement, même la première requête sera déjà complète)


In [18]:
from time import sleep
spark.conf.set("spark.sql.shuffle.partitions", "1")  # keep the size of shuffles small
query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("count3")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)


sql1 = spark.sql("select state_src, date_format(window.end, 'MMM-dd HH:mm') as time, count from count3 order by time,state_src")
sleep(5)  # wait a bit 
sql2 = spark.sql("select state_src, date_format(window.end, 'MMM-dd HH:mm') as time, count from count3 order by time,state_src")
sleep(5)  # wait a bit more
sql3 = spark.sql("select state_src, date_format(window.end, 'MMM-dd HH:mm') as time, count from count3 order by time,state_src")
sleep(5)  # wait a bit more
sql4 = spark.sql("select state_src, date_format(window.end, 'MMM-dd HH:mm') as time, count from count3 order by time,state_src")
query.stop()


In [19]:
z.show(sql1)

In [20]:
z.show(sql2)


In [21]:
z.show(sql3)

In [22]:
z.show(sql4)